In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib
import os

In [3]:
source_dir = "/Users/7408588/Documents/School.nosync/projects/171genomes/ncbi_dataset/data/"

In [ ]:
checkM_cutoff = 99.5
busco_cutoff = 95

df = pd.read_csv(source_dir + 'modded_data_summary.csv')
        curated_genome_frame = df[((df['Level'].isin(['Complete Genome', 'Chromosome']))
                                        | (df['CheckM Score'] > checkM_cutoff) & (df['BUSCO'] > busco_cutoff))]

        linking_dict = dict(zip((df['Assembly Accession'].values.tolist(),
                                 list(np.zeros(curated_genome_frame.shape[0])))))

        def fill_linking_dict():
            """
            Generates a mapping between the accession id and the name of the organism

            Args:
                linking_dict: Mapping between accession id and org name
                source: The source folder that contains the accessions

            Returns:
                None
            """
            with open(self.source_dir + 'assembly_data_report.json') as ifile:
                jsonlist = json.load(ifile)

            for x in jsonlist:
                accession = x['accession']
                orgname = str.split(x['organism']['organismName'], sep=' ')
                strain = x['organism']['infraspecificNames']['strain']

                try:
                    orgname.remove(strain)
                except ValueError:
                    pass

                genus = orgname[0]
                species = orgname[1][:-1] if orgname[-1] == '.' else orgname

                fullname = genus + '_' +  species + '_' + strain

                while fullname[-1] == '_':
                    fullname = fullname[:-1]

                linking_dict.update({accession: fullname})

        fill_linking_dict()
        uniques = np.unique(linking_dict.values(), return_counts=True, return_index=True)

        with open(source_dir + 'curated_genome_list.json', 'w') as ofile:
            json.dump(linking_dict)